In [3]:
import pandas as pd
from typing import Dict, TypedDict
from langgraph.graph import Graph
import numpy as np

In [10]:
class OptimizationState(TypedDict):
    df: pd.DataFrame
    iteration: int
    converged: bool


max_iterations = 5
convergence_threshold = 0.5

def check_convergence(state: OptimizationState) -> str:
    """Check if optimization has converged"""
    if state["iteration"] >= max_iterations or state["converged"]:
        return "end"
    return "continue"

def optimize_step(state: OptimizationState) -> OptimizationState:
    """Perform one optimization step"""
    df = state["df"].copy()
    
    # Simple optimization: move values closer to targets
    df['value'] = df['value'] + 0.1 * (df['target'] - df['value'])
    
    # Check if we've converged
    mean_diff = np.abs(df['value'] - df['target']).mean()
    converged = mean_diff < convergence_threshold
    
    return {
        "df": df,
        "iteration": state["iteration"] + 1,
        "converged": converged
    }


workflow = Graph()

# Add nodes
workflow.add_node("optimize", optimize_step)

# Add edges
workflow.add_edge("optimize", "optimize")

# Set the entry point
workflow.set_entry_point("optimize")

# Set the conditional edges
workflow.add_conditional_edges(
    "optimize",
    check_convergence,
    {
        "continue": "optimize",
        "end": "end"
    }
)

In [5]:
comp_workflow = workflow.compile()

In [11]:

df = pd.DataFrame({
        'value': [10, 15, 20, 25, 30],
        'target': [12, 18, 22, 24, 28]
    })

initial_state = {
            "df": df,
            "iteration": 0,
            "converged": False
        }

result = comp_workflow.invoke(initial_state)

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [9]:
from langgraph.graph import Graph, START, END
from typing import TypedDict

# Define our state type
class NumberState(TypedDict):
    number: int

# Define the function that adds 2
def add_two(state: NumberState) -> NumberState:
    return {"number": state["number"] + 2}

# Create and configure the graph
workflow = Graph()

# Add the node
workflow.add_node("add_two", add_two)

workflow.add_edge(START, "add_two")
workflow.add_edge("add_two", END)

# Compile the graph
chain = workflow.compile()

# Run it with an initial state
initial_state = {"number": 5}
result = chain.invoke(initial_state)

print(f"Initial number: {initial_state['number']}")
print(f"Final number: {result['number']}")

Initial number: 5
Final number: 7
